In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
# 1. Importamos archivo csv
df=pd.read_csv('src/attacks.csv',encoding='latin-1') #Incluir encoding='latin-1'o UTF-8/16 para que pueda abrir el archivo
print(df.shape) # ver cuantas filas/columnas tiene el archivo
df.head()

(25723, 24)


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [3]:
df.columns # ver que columnas hay en el csv

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [4]:
#2. Eliminamos duplicados
df.drop_duplicates()
df.shape

(25723, 24)

In [5]:
#3.Encontrar columnas con valores nulos
df.isnull().head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
1,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,True
2,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,True
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True


In [6]:
#Ver total de filas nulas de cada columna
null_columns=df.isnull().sum()
null_columns[null_columns>0]

Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

In [7]:
#Comprobar que contenido aportan las columnas "Case Number 1 and 2" diferente de la columna "Case Number"
df[["Case Number","Case Number.1","Case Number.2"]] #A primera vista parece que tiene el mismo contenido
df["Case Number.1"].equals(df["Case Number.2"].equals(df['Case Number']))#False
df["Case Number.1"].isin(df["Case Number.2"]).value_counts()# solo 19 False
df["Case Number.1"].isin(df["Case Number"]).value_counts() #Solo 23 False

# Conclusión: las columnas Case Number.1 y Case Number.2 tienen prácticamente el mismo contenido que  Case Number por lo que podemos eliminarlas


True     25700
False       23
Name: Case Number.1, dtype: int64

In [8]:
#Ademas elimino el resto de columnas que no aportan ningun valor al analisis: "pdf","href formula","href","Unnamed: 22","Unnamed: 23","original order","Investigator or Source"

df_clean = df.drop(columns=["Case Number.1","Case Number.2","pdf","href formula","href","Unnamed: 22","Unnamed: 23","original order","Investigator or Source"])
df_clean.shape

(25723, 15)

In [9]:
df_clean.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species '],
      dtype='object')

In [10]:
df_clean.isnull().sum()

Case Number    17021
Date           19421
Year           19423
Type           19425
Country        19471
Area           19876
Location       19961
Activity       19965
Name           19631
Sex            19986
Age            22252
Injury         19449
Fatal (Y/N)    19960
Time           22775
Species        22259
dtype: int64

In [11]:
#Drop the rows where all elements are missing -> al haber tantas rows vacias elimino las filas que tengan todas las columnas nulas
df_clean=df_clean.dropna(how='all')
df_clean.shape

(8703, 15)

In [12]:
df_clean.isnull().sum()

Case Number       1
Date           2401
Year           2403
Type           2405
Country        2451
Area           2856
Location       2941
Activity       2945
Name           2611
Sex            2966
Age            5232
Injury         2429
Fatal (Y/N)    2940
Time           5755
Species        5239
dtype: int64

In [13]:
# Limpio columna "Case Number"
df_clean["Case Number"].value_counts() # Parece que el contenido de la columna es igual a la fecha= Columna Date/Year
df_clean[df_clean["Case Number"].isnull()][["Year","Date"]] # Compruebo la columna nula con columnas relacionadas

,Year,Date
5488,1905.0,Reported 06-Sep-1905


In [14]:
df_clean["Case Number"]= df_clean["Case Number"].fillna("1905.09.06") # Relleno info nula con info de columnas relacionadas

In [15]:
df_clean.isnull().sum()

Case Number       0
Date           2401
Year           2403
Type           2405
Country        2451
Area           2856
Location       2941
Activity       2945
Name           2611
Sex            2966
Age            5232
Injury         2429
Fatal (Y/N)    2940
Time           5755
Species        5239
dtype: int64

In [16]:
# Elimino todas las filas que tengan por lo menos 12 columnas nulas (hay 15 en total)
df_clean=df_clean.dropna(thresh=12)
df_clean.shape

(5343, 15)

In [17]:
df_clean.isnull().sum()

Case Number       0
Date              0
Year              0
Type              1
Country           2
Area            104
Location        144
Activity        155
Name             24
Sex             217
Age            1936
Injury            3
Fatal (Y/N)     365
Time           2430
Species        2144
dtype: int64

In [18]:
# Limpio columna "Type"
df_clean["Type"].value_counts()
df_clean[df_clean["Type"].isnull()][["Activity","Injury","Fatal (Y/N)"]]

,Activity,Injury,Fatal (Y/N)
85,Fishing,Injuries to hands and legs,N


In [19]:
#Añado fila vacia y adapto contenido de la columna para que quede categorizado
df_clean["Type"]= df_clean["Type"].fillna("Boating")
df_clean["Type"]= df_clean["Type"].str.replace("Boatinginging","Boating")
df_clean["Type"]= df_clean["Type"].str.replace("Boatingingomg","Boating")
df_clean["Type"]= df_clean["Type"].str.replace("Questionable","Invalid")
df_clean["Type"]= df_clean["Type"].str.replace("Boatinging","Boating")
df_clean["Type"]= df_clean["Type"].str.replace("Boating","Boating")
df_clean["Type"]= df_clean["Type"].str.replace("Boat","Boating")
df_clean["Type"]= df_clean["Type"].str.replace("Boatingingomg","Boating")
df_clean["Type"]= df_clean["Type"].str.replace("Boatinging","Boating")
df_clean["Type"]= df_clean["Type"].str.replace("Boatingomg","Boating")
df_clean["Type"].value_counts()

Unprovoked      4076
Provoked         520
Invalid          383
Boating          254
Sea Disaster     110
Name: Type, dtype: int64

In [20]:
df_clean.isnull().sum()

Case Number       0
Date              0
Year              0
Type              0
Country           2
Area            104
Location        144
Activity        155
Name             24
Sex             217
Age            1936
Injury            3
Fatal (Y/N)     365
Time           2430
Species        2144
dtype: int64

In [21]:
#Limpio columna "Country"
df_clean["Country"].value_counts()
df_clean[df_clean["Country"].isnull()][["Area","Location","Activity"]]

,Area,Location,Activity
4231,Near the Andaman & Nicobar Islands,NaN,Climbing back on ship
4498,Caribbean Sea,Between Cuba & Costa Rica,"Sea Disaster, sinking of the motorship Wingate"


In [22]:
df_clean.loc[df_clean["Country"].isnull(),"Country"] = ["INDIA","PANAMA"] #Relleno filas nulas
df_clean["Country"]=df_clean["Country"].apply(lambda x: x.lower())# Paso a minusculass
df_clean["Country"]=df_clean["Country"].str.replace('(\w+)', lambda x: x.group(0).capitalize()) # Pongo mayuscula a la primera letra de cada pais
df_clean["Country"].head(10)

0          Usa
1          Usa
2          Usa
3    Australia
4       Mexico
5    Australia
6       Brazil
7          Usa
8          Usa
9          Usa
Name: Country, dtype: object

In [23]:
#Relleno filas nulas del resto de columnas para que quede a 0
df_clean["Area"]= df_clean["Area"].fillna("Unknown")
df_clean["Location"]= df_clean["Location"].fillna("Unknown")
df_clean["Activity"]= df_clean["Activity"].fillna("Unknown")
df_clean["Name"]= df_clean["Name"].fillna("Noname")
df_clean["Sex "]= df_clean["Sex "].fillna("Unknown")
df_clean["Age"]= df_clean["Age"].fillna("0")
df_clean["Injury"]= df_clean["Injury"].fillna("Unknown")
df_clean["Fatal (Y/N)"]= df_clean["Fatal (Y/N)"].fillna("Unknown")
df_clean["Time"]= df_clean["Time"].fillna("Notime")
df_clean["Species "]= df_clean["Species "].fillna("Unknown")

# ALL DATA IS CLEANED
df_clean.isnull().sum()[df_clean.isnull().sum() > 0]

Series([], dtype: int64)

In [24]:
#Verifico tipo de datos en cada columna 
df_clean.dtypes #Todos son object menos Year que es float64

Case Number     object
Date            object
Year           float64
Type            object
Country         object
Area            object
Location        object
Activity        object
Name            object
Sex             object
Age             object
Injury          object
Fatal (Y/N)     object
Time            object
Species         object
dtype: object

In [25]:
#Modifico tipo de column Year
df_clean["Year"]=df_clean["Year"].astype("object")
df_clean["Year"].dtype

dtype('O')

In [26]:
#Corrijo nombres de algunas columnas con espacios
df_clean=df_clean.rename(columns={"Sex ":"Sex"}) #Quito el espacio del nombre de la columna "Sex"
df_clean=df_clean.rename(columns={"Species ":"Species"})

In [27]:
#Limpio contenido columna Activity 
df_clean.loc[df_clean["Activity"].str.contains("wim"),"Activity"]="Swimming"
df_clean.loc[df_clean["Activity"].str.contains("urf"),"Activity"]="Surfing"
df_clean.loc[df_clean["Activity"].str.contains("ish"),"Activity"]="Fishing"
df_clean["Activity"]= df_clean["Activity"].str.replace("Spearfishing","Fishing")
df_clean["Activity"].value_counts().head(10)

Surfing         1225
Fishing         1076
Swimming        1027
Unknown          158
Wading           146
Bathing          143
Diving           102
Standing          98
Snorkeling        84
Scuba diving      75
Name: Activity, dtype: int64

In [47]:
#Limpio contenido columnas relacionadas con fechas:Al estar en mejor formato la columna "Case Number" vamos a limpiar esta para utilizar su contenido
df_clean["Case Number"]=df_clean["Case Number"].map(lambda x: str(x)[0:10]) # quito todos los elementos fuera de la fecha
df_clean.loc[df_clean["Case Number"].str.startswith("ND"),"Case Number"]="0000.00.00" 
df_clean["Case Number"].head(10)

0    2018.06.25
1    2018.06.18
2    2018.06.09
3    2018.06.08
4    2018.06.04
5    2018.06.03
6    2018.06.03
7    2018.05.27
8    2018.05.26
9    2018.05.26
Name: Case Number, dtype: object

In [29]:
# Creo nuevas columnas para diferenciar meses y años de cada fecha en base a columna "Case Number"
df_clean['Day']=[d[8:10] for d in df_clean['Case Number']]
df_clean['Month']=[d[5:7] for d in df_clean['Case Number']]
df_clean["Year2"]=[d[0:4] for d in df_clean['Case Number']]

In [30]:
# Comparo para ver si Year2 y Year tienen mismo contenido
df_clean[["Year2","Year"]].head(20) 
df_clean["Year"].isin(df_clean["Year2"]).value_counts()

False    5343
Name: Year, dtype: int64

In [31]:
# Creo columna Decade para poder categorizar los años
df_clean["Year2"]=df_clean["Year2"].astype("int") # Convierto columna a int para poder crear columna Decade

df_clean["Decade"] = pd.cut(df_clean.Year2,
                                bins=[1800,1818,1838,1858,1878,1898,1918,1938,1958,1978,1998,2018], 
                                labels=range(1800,2020,20))
df_clean["Decade"]= df_clean["Decade"].fillna(1800) # Relleno fila nula (años anteriores a 1908 las incluyo en la categoria 1910)
df_clean["Year2"]=df_clean["Year2"].astype("object") # vuelvo a convertir la columna a object

df_clean["Decade"].value_counts().head(30)

df_clean.isnull().sum()
df_clean["Decade"].value_counts().head(30)

2000    2035
1980     859
1960     858
1940     532
1920     378
1900     242
1880     169
1800      99
1860      96
1840      56
1820      19
Name: Decade, dtype: int64

In [32]:
# Creo nuevo columna de "Hemisphere_South" para categorizar los paises por hemisferio y poder categorizar las seasons
df_clean["Country"].value_counts().head(60)
hemisphere_south=["Australia","South Africa","New Zealand","Papua New Guinea","Brazil","Reunion","Fiji","New Caledonia","Mozambique","French Polynesia","Solomon Islands","Indonesia","Tonga","Vanuatu","Kenya","Chile","Ecuador","Seychelles","Mauritius"]
df_clean["Hemisphere_South"]=np.where(df_clean["Country"].isin(hemisphere_south),True,False)
df_clean[["Hemisphere_South","Country"]].head(10)

,Hemisphere_South,Country
0,False,Usa
1,False,Usa
2,False,Usa
3,True,Australia
4,False,Mexico
5,True,Australia
6,True,Brazil
7,False,Usa
8,False,Usa
9,False,Usa


In [33]:
# Creo nuevo columna de "Seasonality" para categorizar el tipo de season por mes (por hemisferio)
seasons_month = {"01":{True:"Summer",False:"Winter"},
                 "02":{True:"Summer",False:"Winter"},
                 "03":{True:"Autumn",False:"Spring"},
                 "04":{True:"Autumn",False:"Spring"},
                 "05":{True:"Autumn",False:"Spring"},
                 "06":{True:"Winter",False:"Summer"},
                 "07":{True:"Winter",False:"Summer"},
                 "08":{True:"Winter",False:"Summer"},
                 "09":{True:"Spring",False:"Autumn"},
                 "10":{True:"Spring",False:"Autumn"},
                 "11":{True:"Spring",False:"Autumn"},
                 "12":{True:"Summer",False:"Winter"}}

def seasonality(hemi,month):
    return seasons_month.get(month,{}).get(hemi,"Unknown") # hay rows que no coinciden con los elementos del diccionario

df_clean["Seasonality"]=df_clean[["Hemisphere_South","Month"]].apply(lambda row:seasonality(row["Hemisphere_South"],row["Month"]),axis=1)

df_clean[["Seasonality","Country","Month"]].head(10)

,Seasonality,Country,Month
0,Summer,Usa,06
1,Summer,Usa,06
2,Summer,Usa,06
3,Winter,Australia,06
4,Summer,Mexico,06
5,Winter,Australia,06
6,Winter,Brazil,06
7,Spring,Usa,05
8,Spring,Usa,05
9,Spring,Usa,05


In [34]:
#Limpio columna "Fatal (Y/N) para quede categorizada
df_clean["Fatal (Y/N)"].value_counts()
df_clean["Fatal (Y/N)"]= df_clean["Fatal (Y/N)"].str.replace(" N","N")
df_clean["Fatal (Y/N)"]= df_clean["Fatal (Y/N)"].str.replace("2017","Unknown")
df_clean["Fatal (Y/N)"]= df_clean["Fatal (Y/N)"].str.replace("M","Unknown")
df_clean["Fatal (Y/N)"]= df_clean["Fatal (Y/N)"].str.replace("UNKNOWN","Unknown")

In [35]:
df_clean["Fatal (Y/N)"].value_counts()

N          3899
Y          1054
Unknown     390
Name: Fatal (Y/N), dtype: int64

In [36]:
#Limpio columna Injury
df_clean["Injury"]= df_clean["Injury"].str.replace("FATAL","Fatal")
df_clean["Injury"]= df_clean["Injury"].str.replace("Foot bitten","Injury")
df_clean["Injury"]= df_clean["Injury"].str.replace("Leg bitten","Injury")
df_clean.loc[df_clean["Injury"].str.contains("atal"),"Injury"]="Fatal"
df_clean.loc[df_clean["Injury"].str.contains("No injury"),"Injury"]="No injury"
df_clean.loc[df_clean["Injury"].str.contains("Foot"),"Injury"]="Injury"
df_clean.loc[df_clean["Injury"].str.contains("arm"),"Injury"]="Injury"
df_clean.loc[df_clean["Injury"].str.contains("leg"),"Injury"]="Injury"
df_clean.loc[df_clean["Injury"].str.contains("hand"),"Injury"]="Injury"
df_clean.loc[df_clean["Injury"].str.contains("bit"),"Injury"]="Injury"
df_clean.loc[df_clean["Injury"].str.contains("njur"),"Injury"]="Injury"
df_clean["Injury"].value_counts().head(10)

Injury                       2997
Fatal                        1054
Survived                       52
Lacerations to foot            19
Lacerations to left foot       18
Lacerations to right foot      18
No details                     13
Laceration to left foot        10
Thigh lacerated                10
Right foot lacerated            9
Name: Injury, dtype: int64

In [37]:
#Limpio columna "Age"
df_clean["Age"].value_counts()
#Paso todo lo que no es digito a digito
df_clean["Age"]= df_clean["Age"].str.replace("teen","15")
df_clean["Age"]= df_clean["Age"].str.replace("Teen","15")
df_clean["Age"]= df_clean["Age"].str.replace("mid-30s","30")
df_clean["Age"]= df_clean["Age"].str.replace("mid-20s","20")
df_clean["Age"]= df_clean["Age"].str.replace("adult","45")
df_clean["Age"]= df_clean["Age"].str.replace("(adult)","45")
df_clean["Age"]= df_clean["Age"].str.replace("young","22")
df_clean["Age"]= df_clean["Age"].str.replace("(young)","22")
df_clean["Age"]= df_clean["Age"].str.replace("Adult","45")
df_clean["Age"]= df_clean["Age"].str.replace(">50","50")
df_clean["Age"]= df_clean["Age"].str.replace("Elderly","70")

In [38]:
df_clean["Age"].value_counts().head(10)

0     1936
17     151
15     151
18     149
19     140
16     136
20     136
21     118
22     115
25     107
Name: Age, dtype: int64

In [39]:
df_clean[df_clean["Age"].str.match('^(?![0,1,2,3,4,5,6,7,8,9])')] #Busco todas las rowss en las que no aparezcan digitos
df_clean["Age"]=df_clean["Age"].str.extract('(\d+)', expand=False) #Elimino todos los elementos que no tengan digito
df_clean["Age"]= df_clean["Age"].fillna("0")# al generarse nuevas lineas nulas, las reemplazo por 0
df_clean["Age"].value_counts().head(10)

0     1945
17     153
15     152
18     152
20     145
19     141
16     137
21     121
22     116
25     109
Name: Age, dtype: int64

In [40]:
df_clean["Age"]=df_clean["Age"].astype("int") # convierto Age en int para crear nueva categoria "Age Category"

Age_category=[]
for x in df_clean["Age"]:
    if x >= 65:
        Age_category.append("Elderly")
    elif x >= 35:
        Age_category.append("Adults")
    elif x >= 20:
        Age_category.append("Young")
    elif x >= 10:
        Age_category.append("Teens")
    elif 0 > x < 10:
        Age_category.append("Children")
    else:
        Age_category.append("Unknown")

        
df_clean["Age Category"]=Age_category    
df_clean["Age Category"].value_counts().head(10)

Unknown    2063
Young      1290
Teens      1099
Adults      837
Elderly      54
Name: Age Category, dtype: int64

In [41]:
#Categorizamos columna Sex
df_clean["Sex"]= df_clean["Sex"].str.replace(".","Unknown")
df_clean["Sex"]= df_clean["Sex"].str.replace("lli","Unknown")
df_clean["Sex"]= df_clean["Sex"].str.replace("N","Unknown")
df_clean["Sex"]= df_clean["Sex"].str.replace("M ","M")
df_clean["Sex"].value_counts()

M          4536
F           587
Unknown     220
Name: Sex, dtype: int64

In [42]:
#Elimino duplicados
df_clean= df_clean.drop_duplicates()

In [43]:
#Verifico columnas restantes
df_clean.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species', 'Day', 'Month', 'Year2', 'Decade', 'Hemisphere_South',
       'Seasonality', 'Age Category'],
      dtype='object')

In [44]:
#Ordeno orden columnas
column_order = ["Case Number","Date","Year","Time","Country","Area","Location","Type","Activity","Injury","Fatal (Y/N)","Species","Name","Sex","Age","Day","Month","Year2","Decade","Seasonality","Hemisphere_South","Age Category"]
df_clean=df_clean[column_order]

In [51]:
df_analysis = df_clean[["Case Number","Date","Year","Time","Country","Area","Location","Type","Activity","Injury","Fatal (Y/N)","Species","Name","Sex","Age","Day","Month","Year2","Decade","Seasonality","Hemisphere_South","Age Category"]]

In [55]:
df_analysis.shape

(5341, 22)

In [56]:
df_analysis.to_csv("src/attacks_limpio.csv")